In [1]:
# https://sparkbyexamples.com/spark/spark-read-binary-file-into-dataframe/
#df = spark.read.format("image").option("recursiveFileLookup", True).load("s3://multimedia-commons/data/images/{00*,01*}")
df = spark.read.format("image").option("recursiveFileLookup", True).load("s3://multimedia-commons/data/images/{00*}")
df.printSchema()
df.count()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7,application_1650412208045_0009,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |-- width: integer (nullable = true)
 |    |-- nChannels: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- data: binary (nullable = true)

1501

## Use the built-in ```toNDArray``` function

In [2]:
# https://stackoverflow.com/a/69215982/11262633
import pyspark.sql.functions as F
from pyspark.ml.image import ImageSchema
from pyspark.ml.linalg import DenseVector, VectorUDT

@F.udf(returnType=VectorUDT())
def img2vec(x):
    try:
        image_np = DenseVector(ImageSchema.toNDArray(x).flatten())
    except:
        image_np = None
    return image_np

print(f'Image fields = {ImageSchema.imageFields}')
df_new = df.withColumn('vecs',img2vec('image'))
df_new.show()
#df_new.select('vecs').first().asDict().keys()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Image fields = ['origin', 'height', 'width', 'nChannels', 'mode', 'data']
+--------------------+--------------------+
|               image|                vecs|
+--------------------+--------------------+
|{s3://multimedia-...|[201.0,146.0,125....|
|{s3://multimedia-...|[44.0,181.0,150.0...|
|{s3://multimedia-...|[45.0,103.0,108.0...|
|{s3://multimedia-...|[221.0,201.0,190....|
|{s3://multimedia-...|[83.0,80.0,89.0,3...|
|{s3://multimedia-...|[57.0,84.0,94.0,7...|
|{s3://multimedia-...|[187.0,138.0,154....|
|{s3://multimedia-...|[56.0,98.0,81.0,5...|
|{s3://multimedia-...|[0.0,46.0,221.0,0...|
|{s3://multimedia-...|[4.0,82.0,75.0,14...|
|{s3://multimedia-...|[30.0,54.0,90.0,1...|
|{s3://multimedia-...|[254.0,252.0,252....|
|{s3://multimedia-...|[54.0,73.0,81.0,5...|
|{s3://multimedia-...|[85.0,92.0,63.0,8...|
|{s3://multimedia-...|[27.0,25.0,24.0,3...|
|{s3://multimedia-...|[182.0,179.0,158....|
|{s3://multimedia-...|[51.0,86.0,89.0,1...|
|{s3://multimedia-...|[234.0,178.0,96.0...|
|{

In [3]:
df_new.where(df_new.vecs.isNull()).select('image.origin').show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------------------------------------------------------------+
|origin                                                                      |
+----------------------------------------------------------------------------+
|s3://multimedia-commons/data/images/002/7ce/0027ce88ae5dc31b18e3245743e3.jpg|
+----------------------------------------------------------------------------+

In [7]:
image = df_new.where(df_new.image.origin == 's3://multimedia-commons/data/images/002/7ce/0027ce88ae5dc31b18e3245743e3.jpg')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Offending image in DataFrame is 0 Bytes

However, when downloaded, image is 99324 bytes

In [15]:
image_row = image.first()
image_data = image_row.asDict()['image'].asDict()['data']
len(image_data)
print(f'Image size={len(image_data)}')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

In [29]:
image_row

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(image=Row(origin='s3://multimedia-commons/data/images/002/7ce/0027ce88ae5dc31b18e3245743e3.jpg', height=-1, width=-1, nChannels=-1, mode=-1, data=bytearray(b'')), vecs=None)

In [25]:
%%bash
aws s3 cp s3://multimedia-commons/data/images/002/7ce/0027ce88ae5dc31b18e3245743e3.jpg /tmp
ls -l /tmp/0027ce88ae5dc31b18e3245743e3.jpg

download: s3://multimedia-commons/data/images/002/7ce/0027ce88ae5dc31b18e3245743e3.jpg to tmp/0027ce88ae5dc31b18e3245743e3.jpg
-rw-r--r-- 1 emr-notebook emr-notebook 99324 Mar 31  2019 /tmp/0027ce88ae5dc31b18e3245743e3.jpg


In [28]:
#first_image = df_new.first()
first_data = first_image.asDict()['image'].asDict()['data']
print(f'Image {first_image.image.origin} size={len(first_data)}')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Image s3://multimedia-commons/data/images/00d/aa0/00daa0db78a9de9ded0a61dd27fae.jpg size=921600

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3://multimedia-commons/data/images/00d/aa0/00daa0db78a9de9ded0a61dd27fae.jpg'

In [22]:
image_data

ip-172-31-22-251
ERROR: JAVA_HOME is not set and could not be found.


In [16]:
%%local
from PIL import Image
image_local = Image.open('/tmp/ce88ae5dc31b18e3245743e3.jpg')
image_local

FileNotFoundError: [Errno 2] No such file or directory: '/tmp/ce88ae5dc31b18e3245743e3.jpg'

In [38]:
ImageSchema.toNDArray(first_row['image']).shape

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(302, 500, 3)

In [21]:
# https://stackoverflow.com/questions/65713299/javapackage-object-is-not-callable-error-executing-explain-in-pyspark-3-0
from py4j.java_gateway import java_import
java_import(spark._sc._jvm, "org.apache.spark.sql.api.python.*")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
#print(f'# partitions={df_new.rdd.getNumPartitions()}')
df_new.explain()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
*(1) Project [image#96, pythonUDF0#119 AS vecs#105]
+- BatchEvalPython [<lambda>(image#96)], [pythonUDF0#119]
   +- FileScan image [image#96] Batched: false, DataFilters: [], Format: org.apache.spark.ml.source.image.ImageFileFormat@27d52323, Location: InMemoryFileIndex[s3://multimedia-commons/data/images/000, s3://multimedia-commons/data/images/00..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<image:struct<origin:string,height:int,width:int,nChannels:int,mode:int,data:binary>>